# Analiza podatkov
V sledečem zvezku se nahaja analiza podatkov iz spletne strani [Musescore](https://musescore.com/sheetmusic).

V _prvem delu_ bom izvedla preprosto analizo pridobljenih podatkov. 
Zanimalo me bo, kako se posamezni parametri povezujejo s popularnostjo vnosa ter povezave med parametri.
Konkretno želim odgovoriti na naslednja vprašanja:
* Katerih vnosov je največ nasploh in za posamezne kategorije?
* Kateri so najpopularnejši?
* Kakšna je povezava med popularnostjo in številčnostjo nasploh in v posamezni kategoriji?


V _drugem delu_ se bom lotila nekoliko zahtevnejše analize.
Zanimalo me bo, še obstaja recept za uspešno objavo - pri tem bom uporabila pridobljene informacije iz prvega dela; zanimala pa me bo tudi časovna slika podatkov. Tako bom podatke razdelila na smiselne časovne intervale in znotraj teh poiskala parametre, kot je povprečna zvrst, dolžina itn. vnosov v tem obdobju; prav tako pa bom po zastopanosti v tem obdobju uredila originale. Iskala bom obdobja večjih sprememb v podatkih in poskusila najti odgovor, zakaj je do njih prišlo. Zanimala me bo dominantnost posameznih žanrov in inštrumentov, z analizo žanrov pa bom tudi ustvarila sliko glasbenih okusov skozi čas.  
Glede na rezultate bom ustvarila tudi predpostavke za spremembe trendov v prihodnosti.  
Dobljene rezultate bom tudi vizualno predstavila s smiselnimi grafi.  
Na koncu bom rezultate dodala svojemu "receptu" za uspešno objavo in odgovorila na vprašanje, če se ta kaj spremeni.  

## Uvoz podatkov

In [1]:
# Naložimo potrebne knjižnjice
import pandas as pd
import os.path

#Nastavimo izpisovanje po 10 vrstic, saj delamo z ogromno količino podatkov, in bi bile razpredelnice sicer prevelike.
pd.options.display.max_rows = 10

In [2]:
# naložimo osnovno razpredelnico, s katero bomo delali
music = pd.read_csv(os.path.join('obdelani-podatki','all_music.csv'), index_col="music_id")

In [3]:
music

,title,stars,votes,ensemble,instrument,key,date,duration,public_domain,favorites,difficulty,pages,views,parts,link
music_id,,,,,,,,,,,,,,,
4197961,Merry-Go-Round of Life: Howl's Moving Castle P...,5.0,47300,Solo,['Piano'],"['B♭ major', 'G minor']","Jul 05, 2017",05:15,NE,180482,Intermediate,7,4558286,1,https://musescore.com/user/16006641/scores/419...
3291706,River Flows In You,4.5,45200,Solo,['Piano'],"['A major', 'F♯ minor']","Jan 24, 2017",02:47,NE,206915,Intermediate,4,5950761,1,https://musescore.com/user/12461571/scores/329...
1019991,Canon in D,4.5,33700,Solo,['Piano'],"['D major', 'B minor']","Jul 01, 2015",04:05,DA,136613,Intermediate,6,3258924,1,https://musescore.com/user/1809056/scores/1019991
33816,Für Elise – Beethoven,5.0,28900,Solo,['Piano'],"['C major', 'A minor']","Dec 22, 2011",02:37,DA,113864,Intermediate,3,2908874,1,https://musescore.com/user/19710/scores/33816
58553,Clair de Lune – Debussy,5.0,25100,Solo,['Piano'],"['D♭ major', 'B♭ minor']","Aug 05, 2012",04:36,DA,119133,Advanced,6,3243868,1,https://musescore.com/user/19710/scores/58553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5706953,lovely - Billie Eilish ft. Khalid | Piano and ...,4.5,541,"Mixed Duet, Solo","['Violin', 'Piano']","['G major', 'E minor']","Sep 10, 2019",03:16,NE,2550,NaN,6,48304,2,https://musescore.com/user/30386504/scores/570...
3205376,Game of Thrones Theme Song Orchestra,5.0,541,"Chamber Orchestra, Mixed Ensemble","['Flute Piccolo', 'Flute', 'Oboe', 'Clarinet I...","['E♭ major', 'C minor']","Jan 10, 2017",01:34,NE,1084,NaN,5,46647,36,https://musescore.com/user/6596146/scores/3205376
5672113,Jane Maryam - Evgeny Grinko,5.0,540,Solo,['Piano'],"['E♭ major', 'C minor']","Aug 11, 2019",01:30,DA,1392,Beginner,2,70276,1,https://musescore.com/user/7916166/scores/5672113


In [4]:
# Naložimo tudi pomožne razpredelnice, v katerih imamo dodatne podatke
authors = pd.read_csv(os.path.join('obdelani-podatki','authors.csv'), index_col="author_id")
genres = pd.read_csv(os.path.join('obdelani-podatki','genres.csv'))
users = pd.read_csv(os.path.join('obdelani-podatki','users.csv'), index_col="user_id")
relacije_authors = pd.read_csv(os.path.join('obdelani-podatki','relacije_authors.csv'))
relacije_users = pd.read_csv(os.path.join('obdelani-podatki','relacije_users.csv'))

## _Prvi del_: Osnovna analiza

Začnimo našo analizo z obravnavo splošne popularnosti vnosov. Najprej si poglejmo ocene.

In [11]:
min(music.stars)

4.5

Vidimo, da ocen manjših od 4.5 ni, torej te ne bodo zares informativne. Zato nas bo za oceno popularnosti vnosa bolj zanimalo razmerje med ljudmi, ki so vnos videli, in ljudmi, ki so ga dodali med najljubše.